## Version 1.0.0.2 2021 05 04

### Tbot - Trade Bot
 
 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 The code is also hosted at https://github.com/n4rus/tbot and is licensed under creative commons.
 
 Version 0.9.9.7 was the first public version and the main code of tbot in a public repository, therefore the first public version of a project that started in older formats contained in the tbot folder as well. Updates may come in the next months, I really look foward to see how the product of so many headaches can perform. And do not fear, I know It may be simple but even so, publishing in this version is a gift for myself and for the general public, and it save's me the effort of overworking to publish a refined code faster. 

 Version 1.0.0.0 was released! Then I find out it could produce a great bug! :D 
 Version 1.0.0.1 improved the later version without going throught the possible overflow errors or file corruptions it could cause while modifying the default patterns that where preset for the first tests.
 Version 1.0.0.2 this version is the first to acknowledge the overflow error and the file it produced in order to solve for the problem. The later proposed refinements shall be uploaded on this version being later redifened at version 1.0.0.3.

### Library Imports
The first thing we need to do is import the python open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import datetime
import time
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient
from datetime import datetime
from datetime import date
from time import time

nova_client = NovaClient(AccessKey, SecretKey)

### Importing List of currencies symbols, making the API Call and create the table Dataframe
List of crypto-currencies symbols, account balance, and table creation.

In [2]:
#symbols
#symbol = str(input("Enter the symbol as it is in the api  symbol list:"))
#symbol = BTC_USDT'
symbol = 'DOGE_USDT'
symbols = ['DOGE_USDT']

In [3]:
#b
#balance dataframe
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bs = bs + 1 

p = 0
subids = {}
sublcs = {}

sublcq = nova_client.subs()
sublcd = sublcq['data']

blccolumns = ['Currency', 'Sub-Balance']
blcdataframe = pd.DataFrame(columns = bcolumns,)

for i in sublcd:
    sublcd = sublcq['data']
    sub = sublcd[p]
    sublci = sub['subId']
    sublcy = sub['subIdentify']
    subids[f"{sublcy}"] = f"{sublci}"
    subId = sublci
    sublcid = nova_client.subs_balance(subId)
    subd = sublcid['data']
    sd = subd[p]
    subcy = sd['currency']
    sublc = sd['balance']
    sublcs[f"{subcy}"] = f"{sublc}"
p = p + 1
#b
bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)

blcdataframe.append(
        pd.Series([
            subcy,
            sublc,
        ], index = blccolumns),
        ignore_index = True)
blcdataframe = blcdataframe.append(
    pd.Series([
        subcy,
        sublc,
    ], index=blccolumns),
        ignore_index=True)

blcdataframe.sort_values('Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)
bdataframe
bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
blcdataframe

,Currency,Balance,Sub-Balance
0,ETH,NaN,0


### Dolar price, timestamp
Dolar price in terms of BRL using ETH

In [4]:
#ether/dolar dataframe 
#e
e_columns = ['Symbol', 'Ask','Bid', 'Last Price','AVG Price', 'Shares to Buy','High 24h', 'Min 24h', 'Avg 24h', 'Timestamp', 'Requisition Timestamp']
e_dataframe = pd.DataFrame(columns = e_columns)

ticker_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
fdaty = requests.get(ticker_url).json()

y = 0
for a in ticker_url[:1]:
    S = 0
    fS = str(S)
    for i in ticker_url:
        fdata = fdaty['data']
        price = float(fdata['lastPrice'])
        ask = float(fdata['ask'])
        bid = float(fdata['bid'])
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        high = float(fdata['high24h'])
        mini = float(fdata['low24h'])
        mid = float((high - mini)/2) 
        abl = float((ask + bid + price) / 3)
    
        etsp = (fdata['timestamp'])
        tspmi = float(etsp/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime(f"%Y%m%d %H:%M:S")
    
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime(f"%Y%m%d %H:%M:S")
        
        y = y + 1
#e

In [5]:
#side sell/buy dataframe
#s
s_columns = ['Symbol','Side', 'Last Price', 'Shares to Buy', 'Timestamp', 'Requisition Timestamp']
s_dataframe = pd.DataFrame(columns = s_columns)
sdaty = requests.get(ticker_url).json()
sdat = nova_client.list_trades(symbol, limit = 10)

for a in sdat:
    yz = int(0)
    for i in sdat['data']:
        daty = sdat['data']
        data = daty[yz]
        price = float(data['price'])
        side = data['side']
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
    
        etsp = (data['timestamp'])
        tspmi = float(etsp/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime("%Y%m%d %H:%M:%S")
    
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S") 
        yz = yz + 1
#s  

### Creating the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [6]:
#momentum dataframe
#m
listtrades = nova_client.list_orders(symbol, 'FINISHED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
l_columns = ['Symbol', 'B/S', 'AVG Price', 'Momentum AVG Price', 'Momentum','Ask', 'Bid', 'Last Price', 'Ask Bid Last AVG', 'Shares to Buy','Min 24h', 'Avg 24h', 'High 24h', 'Timestamp', 'Requisition Timestamp']
m_dataframe = pd.DataFrame(columns = l_columns)
l_url = nova_client.get_ticker(symbol)
rdat = nova_client.list_trades(symbol, limit = 10)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        fdat = fdaty['data']
        price = fdat['lastPrice']
        bid = fdata['bid']
        ask = fdata['ask']
        high = float(fdata['high24h'])
        mini = float(fdata['low24h'])
        mid = float((high - mini)/2) 
        abl = float((float(ask) + float(bid) + float(price)) / 3)
        
        V = price
        vx = float(data['price'])
        M = float(V) - float(vx)
        momentum = float(vx) - float(price)
        tspmi = float(ldc['timestamp']/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")       
        
        s = s + 1
#m

In [7]:
#order dataframe
#o
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
o_columns = ['Symbol', 'Amount', 'Ticker Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp', 'Requisition Timestamp']
o_dataframe = pd.DataFrame(columns = o_columns)
tdaty = requests.get(ticker_url).json()
tdat = nova_client.list_trades('BTC_BRL', limit = 10)
sf = 0
for i in ttrades:
    listorders = nova_client.list_orders(symbol, 'FILLED')
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[sf]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        vxdata = tdata[sf]
        vxi = vxdata['price']
        momentumprice = float(vx)
        momentumraw = float(price) - float(vxi)
        
        V = price
        vx = float(data['price'])
        M = float(V) - float(vx)
        momentum = float(vx) - float(price)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")
sf = sf + 1
#o

In [8]:
#Data-frame assembly

#e
e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        ask,
                        bid,
                        price,
                        abl,
                        shares_to_buy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver
                    ]
                    ,index = e_columns),
                ignore_index=True)
           
e_dataframe = e_dataframe.append(
            pd.Series([
                        symbol, 
                        ask,
                        bid,
                        price,
                        abl,
                        shares_to_buy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver
                        ],
                    index = e_columns
                    ),
                    ignore_index = True)
    
e_dataframe.reset_index(drop = True, inplace = True)
e_dataframe.sort_values('Timestamp', ascending = True, inplace = True)
#e

#s
s_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        side,
                        price,
                        shares_to_buy,
                        tspm,
                        tserver
                    ]
                    ,index = s_columns),
                ignore_index=True)
           
s_dataframe = s_dataframe.append(
            pd.Series([
                        symbol, 
                        side,
                        price,
                        shares_to_buy,
                        tspm,
                        tserver
                        ],
                    index = s_columns
                    ),
                    ignore_index = True)
    
s_dataframe.reset_index(drop = True, inplace = True)
s_dataframe.sort_values('Timestamp', ascending = True, inplace = True)
#s

#m
m_dataframe.append(
                    pd.Series(
                    [
                        symbol, 
                        bs,
                        vx,
                        momentum,
                        M,
                        ask,
                        bid,
                        price,
                        abl,
                        sharestobuy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver    
                    ]
                    ,index = l_columns),
                ignore_index=True)

    
m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol, 
                        bs,
                        vx,
                        momentum,
                        M,
                        ask,
                        bid,
                        price,
                        abl,
                        sharestobuy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver                 
                      ]
                ,index = l_columns), 
                ignore_index = True)
        
m_dataframe.sort_values('Timestamp', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
#m

#o
o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp,
                        tserver,
                    ]
                    ,index = o_columns),
                ignore_index=True)

o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp,
                        tserver,
                ]
                ,index = o_columns), 
                ignore_index = True)

o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
#o

In [9]:
o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

,Symbol,Amount,Ticker Price,B/S,Momentum Price,momentum-raw-data,Status,Timestamp,Requisition Timestamp
0,DOGE_USDT,59.87,0.5577,BUY,0.64985,-0.09861,FILLED,20210504 16:17:22,20210505 01:40:20


In [10]:
#boolean dataframe from orders dataframe[ok]
my_columns = ['Symbol','B/S', 'Amount', 'Ticker Price', 'Momentum-Price',  'Momentum Profit', 'Status', 'Timestamp', 'Requisition Timestamp']
boo_dataframe = pd.DataFrame(columns = my_columns)

ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']

api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
fdaty = requests.get(api_url).json()

fdat = nova_client.list_trades(symbol, limit = 10)

for symbol in symbols:
    listorders = nova_client.list_orders(symbol, 'FILLED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        bs = ldc['side']
        
        vxdata = tdata[s]
        vxi = float(vxdata['price'])
        momentump = float(price*2) - float(vxi)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")
             
        s = s + 1
        
        boo_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        bs,
                        amount,
                        price,
                        vxi,
                        momentump,
                        status,
                        tsp,
                        tserver,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        boo_dataframe = boo_dataframe.append(
                pd.Series([
                        symbol,
                        bs,
                        amount,
                        price,
                        vxi,
                        momentump,
                        status,
                        tsp,
                        tserver,
                      ]
                ,index = my_columns), 
                ignore_index = True)

boo_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
boo_dataframe.reset_index(drop = True, inplace = True)
boo_dataframe

,Symbol,B/S,Amount,Ticker Price,Momentum-Price,Momentum Profit,Status,Timestamp,Requisition Timestamp
0,DOGE_USDT,BUY,59.87,0.55770,0.65631,0.45909,FILLED,20210504 16:17:22,20210505 01:40:24
1,DOGE_USDT,SELL,126.07,0.42348,0.64663,0.20033,FILLED,20210503 23:15:52,20210505 01:40:24
2,DOGE_USDT,BUY,126.07,0.42655,0.66915,0.18395,FILLED,20210503 23:10:46,20210505 01:40:24
3,DOGE_USDT,SELL,64.08,0.44258,0.67116,0.21400,FILLED,20210503 20:35:27,20210505 01:40:24
4,DOGE_USDT,SELL,64.09,0.43244,0.67530,0.18958,FILLED,20210503 18:24:31,20210505 01:40:24
5,DOGE_USDT,BUY,63.25,0.41050,0.67470,0.14630,FILLED,20210503 13:18:00,20210505 01:40:24
6,DOGE_USDT,BUY,64.92,0.41020,0.67220,0.14820,FILLED,20210503 13:15:47,20210505 01:40:24
7,DOGE_USDT,SELL,72.44,0.32990,0.67052,-0.01072,FILLED,20210430 18:59:09,20210505 01:40:24
8,DOGE_USDT,BUY,72.44,0.30620,0.66770,-0.05530,FILLED,20210429 14:25:10,20210505 01:40:24


In [11]:
#boolean dataframe init
def timemachine(x):
    k = 5
    #is different from localtime by k hours
    localtimer = int(time()) + k + 360
    while x < localtimer:
        x = x + 1 + localtimer
        return x
#auto cicle n minutes
j= 0
tmsd = []
ui = math.floor(int(input("Enter the amount of minutes to keep alive loop: ")))
n = 1+ui
z = 0
for i in boo_dataframe:
    if j < 60:
        while j < 60:
            tj = timemachine(j)
            tms = datetime.fromtimestamp(tj).strftime("%Y%m%d %H:%M:%S")
            tmsd.append(tms)
            j = j + 1
    if j == 60:
            j = j - 59
            m = z + 1
    if m == n:
        break

Enter the amount of minutes to keep alive loop: 15


In [12]:
#auto cicle e redo boolean dataframe[1/2ok-1.0.0.1-ct]
rd_columns = ['Symbol','B/S', 'Amount', 'Ticker Price', 'Momentum-Price', 'Momentum Profit', 'Status', 'Timestamp', 'Requisition Timestamp']
rd_dataframe = pd.DataFrame(columns = my_columns)
rdtrades = nova_client.list_trades(symbol, limit=9)
rddata = ttrades['data']
rd_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
rddaty = requests.get(rd_url).json()

rddat = nova_client.list_trades(symbol, limit = 10)
yr = 1
tmsy = tmsd[yr]
xz = dl
d_l = 0
timemachine(xz)
dead_dict = {}

#sub booleab
sublcid = nova_client.subs_balance(subId)
subd = sublcid['data']
blcdataframe['Indexes'] = blcdataframe['Currency'].str.find('ETH')

ct = 0
for i in blcdataframe['Indexes']:  
    indexes = blcdataframe['Indexes']
    idx = indexes[ct]
    if idx == 0:
        osublc = blcdataframe['Sub-Balance']
        oblc = osublc[ct]
        print(oblc)
    ct = ct +1


for i in tmsy:
    listorders = nova_client.list_orders(symbol, 'FILLED')
    s = 0
    for i in listorders:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        vxdata = tdata[s]
        vxi = float(vxdata['price'])
        momentump = float(price*2) - float(vxi)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")
        
        sell = 0
        buy = 0
        sm = 0
        bm = 0
        dead_loop = []
        
        aps = float(oblc)
        apsf = float((aps//0.0001 / 1000))
        profit = float((price/100)*2) + price
        profitm = round(profit, 2)
     
        #SELL CONDITIONALS
        if float(oblc) > 0.02:
        #HIGH MOMENTUM
            if vxi >= profitm:
                result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = str(profitm), amount = str(apsf))
                #print("Sold ", apsf, subcy, " for ", profitm)
        #LOW MOMENTUM
            if vxi < profitm:                                           
                lwm = 0
                while vx < profitm:
                    #result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profitm, amount = apsf)
                    dead_dict['\r',]=[lwm, '''\r''']
                    lwm=lwm+1
        if float(oblc) < 0.02:
        #BUY CONDITIONALS
        #LOW MOMENTUM
            if vxi < price:
                result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = profitm, amount = sublc)
                #print("There was no balance.", symbol, " was added for ", ap) 
        #HIGH MOMENTUM        
        #NO BALLANCE               
            if vxi < price and amount > 0.02 or bs == 'SELL':
                d_l = d_l + 1
                dead_loop_info = f"Dead Loop{d_l}. There was no enought {symbol} balance. The total amount is , {oblc}." 
                dead_loop.append(dead_loop_info) 
                #print(dead_loop)
        s = s + 1
        yr = yr + 1
        
        rd_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        bs,
                        amount,
                        price,
                        vxi,
                        momentump,
                        status,
                        tsp,
                        tmsy,
                    ]
                    ,index = rd_columns),
        ignore_index=True)

    
        rd_dataframe = rd_dataframe.append(
                pd.Series([
                        symbol,
                        bs,
                        amount,
                        price,
                        vxi,
                        momentump,
                        status,
                        tsp,
                        tmsy,
                      ]
                ,index = my_columns), 
                ignore_index = True)

rd_dataframe.sort_values('Requisition Timestamp', ascending = False, inplace = True)
rd_dataframe.reset_index(drop = True, inplace = True)
rd_dataframe

0


,Symbol,B/S,Amount,Ticker Price,Momentum-Price,Momentum Profit,Status,Timestamp,Requisition Timestamp
0,DOGE_USDT,BUY,59.87,0.55770,0.65631,0.45909,FILLED,20210504 16:17:22,20210505 01:46:50
1,DOGE_USDT,BUY,126.07,0.42655,0.66915,0.18395,FILLED,20210503 23:10:46,20210505 01:46:50
2,DOGE_USDT,SELL,126.07,0.42348,0.64663,0.20033,FILLED,20210503 23:15:52,20210505 01:46:50
3,DOGE_USDT,BUY,126.07,0.42655,0.66915,0.18395,FILLED,20210503 23:10:46,20210505 01:46:50
4,DOGE_USDT,BUY,59.87,0.55770,0.65631,0.45909,FILLED,20210504 16:17:22,20210505 01:46:50
5,DOGE_USDT,SELL,126.07,0.42348,0.64663,0.20033,FILLED,20210503 23:15:52,20210505 01:46:50
6,DOGE_USDT,BUY,126.07,0.42655,0.66915,0.18395,FILLED,20210503 23:10:46,20210505 01:46:50
7,DOGE_USDT,BUY,59.87,0.55770,0.65631,0.45909,FILLED,20210504 16:17:22,20210505 01:46:50
8,DOGE_USDT,SELL,126.07,0.42348,0.64663,0.20033,FILLED,20210503 23:15:52,20210505 01:46:50
9,DOGE_USDT,BUY,126.07,0.42655,0.66915,0.18395,FILLED,20210503 23:10:46,20210505 01:46:50


In [13]:
###BaD LOGIG GROUP redo conditionals[ok]
#SELL CONDITIONALS[ok]

#BUY CONDITIONALS[ok]

#HIGH MOMENTUM[ok]

#LOW MOMENTUM[ok]

#HAS BALLANCE[ok]

In [14]:
###Task list
#loop time count and do momentum profit[ok]
#publish[ok]
#add auto cicle[ok]
#redo bad logic group o[ok] s[] r[] (order, simplify, refine)
#simplify the main code and exclude repeated lines

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [15]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
boo_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)
rd_dataframe.to_excel(writer, sheet_name = 'redoboolist', index = False)

In [16]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [17]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Buy/Sell', string_template],
    'C':['Amount', float_template],
    'D':['Last Orders Price', dollar_template],
    'F':['Momentum Price', dollar_template],
    'G':['Profit', dollar_template],
    'H':['Status', string_template],
    'I':['Timestamp', float_template],
    'J':['Requisition Timestamp', float_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [18]:
writer.save()